In [2]:
import cv2
import csv
import keras 
import numpy as np 
from matplotlib import image
from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import Lambda
from keras.models import Model
from keras.layers import Dropout
from keras.regularizers import l2

In [3]:
lines = []
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
images = []
measurements = []
for line in lines:
    #Read the path from the file
    img_center = line[0]
    img_left = line[1]
    img_right = line[2]
    filename_center = img_center.split('/')[-1]
    filename_center = 'data/IMG/'+ filename_center
    filename_left = img_left.split('/')[-1]
    filename_left = 'data/IMG/'+ filename_left
    filename_right = img_right.split('/')[-1]
    filename_right = 'data/IMG/'+ filename_right
    #From image to array
    image_center = image.imread(filename_center)
    image_left = image.imread(filename_left)
    image_right =  image.imread(filename_right)
    image_flipped = np.fliplr(image_center)          #Flip image
    
    images.append(image_center)
    images.append(image_left)
    images.append(image_right)
    images.append(image_flipped)
    
    #Read angle of rotation from the file
    correction = 0.15                                #Image correction from the left and right camera
    steering_center = float(line[3])
    steering_left = steering_center + correction
    steering_right = steering_center - correction
    steering_center_fliped = -steering_center
    measurements.append(steering_center)
    measurements.append(steering_left)
    measurements.append(steering_right)
    measurements.append(steering_center_fliped)

X_train = np.array(images)
y_train = np.array(measurements)
print(X_train.shape)
print(y_train.shape)

(32144, 160, 320, 3)
(32144,)


In [4]:
input_shape = (160,320,3)
inputs = Input(shape=input_shape)
# Crop images
x = keras.layers.convolutional.Cropping2D(cropping=((50,20),(0,0)),input_shape = input_shape)(inputs)
# Normalize images
x = Lambda(lambda x: x / 255.0 - 0.5, input_shape = input_shape)(x)

x = Conv2D(24, (5, 5), strides=(2, 2),  activation='relu', kernel_regularizer=l2(0.0001))(x)
x = Conv2D(36, (5, 5), strides=(2, 2),  activation='relu', kernel_regularizer=l2(0.0001))(x)
x = Conv2D(48, (5, 5), strides=(2, 2),  activation='relu', kernel_regularizer=l2(0.0001))(x)
x = Conv2D(64, (3, 3), strides=(1, 1),  activation='relu', kernel_regularizer=l2(0.0001))(x)
x = Conv2D(64, (3, 3), strides=(1, 1),  activation='relu', kernel_regularizer=l2(0.0001))(x)
x = Flatten()(x)
x = Dense(100)(x)
x = Dense(50)(x)
x = Dropout(0.5)(x)
x = Dense(10)(x)
x = Dense(1)(x)


In [5]:
model = Model(inputs, x, name='Nvidia_self')
model.compile(loss='mse',
              optimizer='Adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,validation_split = 0.2, shuffle=True, nb_epoch=5)
model.save('model.h5')

c:\users\slava\anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 25715 samples, validate on 6429 samples
Epoch 1/5
25715/25715 [==============================] - 36s - loss: 0.0214 - acc: 0.2702 - val_loss: 0.0148 - val_acc: 0.2758
Epoch 2/5
25715/25715 [==============================] - 34s - loss: 0.0142 - acc: 0.2702 - val_loss: 0.0143 - val_acc: 0.2758
Epoch 3/5
25715/25715 [==============================] - 34s - loss: 0.0130 - acc: 0.2702 - val_loss: 0.0147 - val_acc: 0.2758
Epoch 4/5
25715/25715 [==============================] - 35s - loss: 0.0125 - acc: 0.2703 - val_loss: 0.0137 - val_acc: 0.2758
Epoch 5/5
25715/25715 [==============================] - 34s - loss: 0.0122 - acc: 0.2702 - val_loss: 0.0142 - val_acc: 0.2758
